1. Funções de ativação 

In [1]:
import numpy as np
import pandas as pd

# Funções de ativação
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivada_sigmoid(x):
    return x * (1 - x)

2. Classe Neuronio

In [2]:
# Classe Neurônio
class Neuronio:
    def __init__(self, num_entradas, funcao_ativacao, derivada_ativacao):
        self.pesos = np.random.randn(num_entradas) * np.sqrt(2.0 / num_entradas)  # Inicialização Xavier
        self.bias = np.random.randn() * 0.01
        self.funcao_ativacao = funcao_ativacao
        self.derivada_ativacao = derivada_ativacao
        self.saida = None
        self.delta = None

    def calcular_saida(self, entradas):
        self.entradas = entradas
        soma_ponderada = np.dot(self.entradas, self.pesos) + self.bias
        self.saida = self.funcao_ativacao(soma_ponderada)
        return self.saida

    def atualizar_pesos(self, taxa_aprendizado, entrada_anterior):
        self.pesos += taxa_aprendizado * entrada_anterior * self.delta
        self.bias += taxa_aprendizado * self.delta

3. Classe Camada:
   Uma camada é um conjunto de neurônios.

In [3]:
# Classe Camada
class Camada:
    def __init__(self, num_neuronios, num_entradas, funcao_ativacao, derivada_ativacao):
        self.neuronios = [Neuronio(num_entradas, funcao_ativacao, derivada_ativacao) for _ in range(num_neuronios)]
        self.saidas = None
        self.entradas = None
        self.delta = None

    def forward(self, entradas):
        self.entradas = entradas
        self.saidas = np.array([neuronio.calcular_saida(entradas) for neuronio in self.neuronios])
        return self.saidas

    def backward(self, erro_camada_posterior, pesos_camada_posterior, derivada_ativacao_saidas):
        delta_atual = np.dot(erro_camada_posterior, pesos_camada_posterior) * derivada_ativacao_saidas
        self.delta = delta_atual
        for i, neuronio in enumerate(self.neuronios):
            neuronio.delta = delta_atual[i]
        return delta_atual

    def atualizar_pesos(self, taxa_aprendizado, entradas_camada_anterior):
        for neuronio in self.neuronios:
            neuronio.atualizar_pesos(taxa_aprendizado, entradas_camada_anterior)

    def obter_pesos(self):
        return np.array([neuronio.pesos for neuronio in self.neuronios])

4. Classe MLP:

   A rede MLP é composta por várias camadas.

In [4]:
# Classe MLP
class MLP:
    def __init__(self, num_entradas, arquitetura, funcao_ativacao, derivada_ativacao):
        self.camadas = []
        num_neuronios_anterior = num_entradas
        for num_neuronios in arquitetura:
            self.camadas.append(Camada(num_neuronios, num_neuronios_anterior, funcao_ativacao, derivada_ativacao))
            num_neuronios_anterior = num_neuronios

    def forward(self, entrada):
        saida_atual = entrada
        for camada in self.camadas:
            saida_atual = camada.forward(saida_atual)
        return saida_atual

    def backward(self, saida_esperada, saida_predita):
        erro = saida_esperada - saida_predita
        num_camadas = len(self.camadas)
        for i in reversed(range(num_camadas)):
            camada_atual = self.camadas[i]
            if i == num_camadas - 1:  # Camada de saída
                delta_atual = erro * derivada_sigmoid(camada_atual.saidas)
                for j, neuronio in enumerate(camada_atual.neuronios):
                    neuronio.delta = delta_atual[j]
            else:  # Camadas ocultas
                camada_posterior = self.camadas[i + 1]
                pesos_camada_posterior = camada_posterior.obter_pesos()
                delta_atual = camada_atual.backward(delta_atual, pesos_camada_posterior, derivada_sigmoid(camada_atual.saidas))
                for j, neuronio in enumerate(camada_atual.neuronios):
                    neuronio.delta = delta_atual[j]
            camada_atual.delta = delta_atual

    def atualizar_pesos(self, taxa_aprendizado, entrada):
        entrada_anterior = entrada
        for camada in self.camadas:
            camada.atualizar_pesos(taxa_aprendizado, entrada_anterior)
            entrada_anterior = camada.saidas

    def treinar(self, dados_treinamento, rotulos_treinamento, taxa_aprendizado, num_epocas):
        for epoca in range(num_epocas):
            perda_total = 0
            for i, entrada in enumerate(dados_treinamento):
                saida_predita = self.forward(entrada)
                saida_esperada = rotulos_treinamento[i]
                perda = np.mean((saida_esperada - saida_predita) ** 2)
                perda_total += perda
                self.backward(saida_esperada, saida_predita)
                self.atualizar_pesos(taxa_aprendizado, entrada)
            print(f"Época {epoca + 1}/{num_epocas}, Perda: {perda_total / len(dados_treinamento):.6f}")

    def predizer(self, dados_teste):
        predicoes = []
        for entrada in dados_teste:
            saida = self.forward(entrada)
            predicao = np.argmax(saida)
            predicoes.append(predicao)
        return predicoes

    def obter_pesos_treinados(self):
        pesos_treinados = []
        for i, camada in enumerate(self.camadas):
            pesos_camada = camada.obter_pesos()
            bias_camada = np.array([neuronio.bias for neuronio in camada.neuronios])
            pesos_treinados.append((f"Camada {i+1}", pesos_camada, bias_camada))
        return pesos_treinados

5. Carregar os Dados e pré processar os dados.

In [5]:
# Funções de carregamento e pré-processamento
def carregar_dados(nome_arquivo):
    try:
        df = pd.read_csv(nome_arquivo, header=None)
        dados = df.iloc[:, :-1].values.astype(float)  # Características (A, B, C, D)
        rotulos = df.iloc[:, -1].values  # Classes (E)
        return dados, rotulos
    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo}' não encontrado.")
        return None, None

def normalizar_dados(dados):
    minimos = np.min(dados, axis=0)
    maximos = np.max(dados, axis=0)
    range_valores = maximos - minimos
    range_valores[range_valores == 0] = 1
    return (dados - minimos) / range_valores

def codificar_rotulos(rotulos):
    classes = np.unique(rotulos)
    num_classes = len(classes)
    mapeamento = {classe: i for i, classe in enumerate(classes)}
    rotulos_codificados = np.zeros((len(rotulos), num_classes))
    for i, rotulo in enumerate(rotulos):
        indice_classe = mapeamento[rotulo]
        rotulos_codificados[i, indice_classe] = 1
    return rotulos_codificados, classes

6. Normalizar e Codificar Rótulos

In [6]:
# Programa principal
if __name__ == '__main__':
    # Carregar dados
    dados_treinamento, rotulos_treinamento_texto = carregar_dados('iris-train.data')
    dados_teste, rotulos_teste_texto = carregar_dados('iris-test.data')

    if dados_treinamento is not None and dados_teste is not None:
        print("Dados carregados com sucesso!")
        print(f"Número de instâncias de treinamento: {len(dados_treinamento)}")
        print(f"Número de instâncias de teste: {len(dados_teste)}")

        # Normalizar dados
        dados_treinamento_normalizados = normalizar_dados(dados_treinamento)
        dados_teste_normalizados = normalizar_dados(dados_teste)
        print("Dados normalizados com sucesso!")
        print(f"Primeiras 5 instâncias de treinamento normalizadas:\n{dados_treinamento_normalizados[:5]}")
        print(f"Primeiras 5 instâncias de teste normalizadas:\n{dados_teste_normalizados[:5]}")

        # Codificar rótulos
        rotulos_treinamento, classes_treinamento = codificar_rotulos(rotulos_treinamento_texto)
        rotulos_teste, classes_teste = codificar_rotulos(rotulos_teste_texto)
        print("Rótulos codificados com sucesso!")
        print(f"Mapeamento de classes: {classes_treinamento}")
        print(f"Primeiros 5 rótulos de treinamento codificados:\n{rotulos_treinamento[:5]}")
        print(f"Primeiros 5 rótulos de teste codificados:\n{rotulos_teste[:5]}")

        if not np.array_equal(classes_treinamento, classes_teste):
            print("Aviso: As classes nos conjuntos de treinamento e teste são diferentes!")

Dados carregados com sucesso!
Número de instâncias de treinamento: 120
Número de instâncias de teste: 30
Dados normalizados com sucesso!
Primeiras 5 instâncias de treinamento normalizadas:
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]
Primeiras 5 instâncias de teste normalizadas:
[[0.16       0.46666667 0.02173913 0.04347826]
 [0.28       1.         0.06521739 0.        ]
 [0.08       0.6        0.02173913 0.        ]
 [0.36       0.93333333 0.04347826 0.        ]
 [0.24       0.66666667 0.02173913 0.        ]]
Rótulos codificados com sucesso!
Mapeamento de classes: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
Primeiros 5 rótulos de treinamento codificados:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Primeiros 5 rótulos de teste codificados:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]

8. Instanciar a Rede MLP e Treinar

Agora vamos definir a arquitetura da nossa rede e instanciá-la. A arquitetura é uma lista que especifica o número de neurônios em cada camada oculta. Para começar, podemos tentar com uma única camada oculta com um certo número de neurônios (por exemplo, 8). Para a camada de saída, teremos 3 neurônios (um para cada classe). Vamos usar a função sigmoide como função de ativação para as camadas ocultas e a função softmax para a camada de saída.
Vamos treinar a rede usando os dados de treinamento normalizados e os rótulos codificados. Precisamos definir a taxa de aprendizado e o número de épocas.

In [7]:
        # Configurar e treinar a MLP
        num_entradas = 4  # Comprimento e largura da sépala e pétala
        arquitetura = [8, 3]  # Camada oculta com 8 neurônios, saída com 3 neurônios
        mlp = MLP(num_entradas, arquitetura, sigmoid, derivada_sigmoid)

        taxa_aprendizado = 0.01  # Ajustado para convergência
        num_epocas = 1000  # Aumentado para melhorar acurácia

        print("\nIniciando o treinamento da rede...")
        mlp.treinar(dados_treinamento_normalizados, rotulos_treinamento, taxa_aprendizado, num_epocas)
        print("Treinamento da rede concluído!")


Iniciando o treinamento da rede...
Época 1/1000, Perda: 0.262892
Época 2/1000, Perda: 0.238969
Época 3/1000, Perda: 0.223975
Época 4/1000, Perda: 0.215615
Época 5/1000, Perda: 0.211041
Época 6/1000, Perda: 0.208399
Época 7/1000, Perda: 0.206715
Época 8/1000, Perda: 0.205510
Época 9/1000, Perda: 0.204545
Época 10/1000, Perda: 0.203699
Época 11/1000, Perda: 0.202911
Época 12/1000, Perda: 0.202149
Época 13/1000, Perda: 0.201396
Época 14/1000, Perda: 0.200642
Época 15/1000, Perda: 0.199883
Época 16/1000, Perda: 0.199116
Época 17/1000, Perda: 0.198340
Época 18/1000, Perda: 0.197555
Época 19/1000, Perda: 0.196760
Época 20/1000, Perda: 0.195956
Época 21/1000, Perda: 0.195143
Época 22/1000, Perda: 0.194320
Época 23/1000, Perda: 0.193489
Época 24/1000, Perda: 0.192649
Época 25/1000, Perda: 0.191800
Época 26/1000, Perda: 0.190943
Época 27/1000, Perda: 0.190079
Época 28/1000, Perda: 0.189207
Época 29/1000, Perda: 0.188328
Época 30/1000, Perda: 0.187442
Época 31/1000, Perda: 0.186549
Época 32/100

9. Avaliar a rede

In [8]:
        # Avaliar no conjunto de teste
        predicoes_teste = mlp.predizer(dados_teste_normalizados)
        acuracia = np.mean(predicoes_teste == np.argmax(rotulos_teste, axis=1))
        print(f"\nAcurácia da rede nos dados de teste: {acuracia * 100:.2f}%")


Acurácia da rede nos dados de teste: 100.00%


10. Exibir os Resultados.

In [9]:
        # Exibir pesos treinados
        print("\nPesos da rede treinada:")
        pesos_treinados = mlp.obter_pesos_treinados()
        for nome_camada, pesos, bias in pesos_treinados:
            print(f"{nome_camada}:")
            print("  Pesos:")
            print(pesos)
            print("  Bias:")
            print(bias)

        # Exibir classificação das instâncias de teste
        print("\nClassificação das instâncias dos dados de teste:")
        for i, entrada in enumerate(dados_teste_normalizados):
            predicao = predicoes_teste[i]
            classe_predita = classes_teste[predicao]
            classe_real = rotulos_teste_texto[i]
            print(f"Instância {i + 1}: Características={entrada}, Predito={classe_predita}, Real={classe_real}")


Pesos da rede treinada:
Camada 1:
  Pesos:
[[ 1.15016893 -0.21384892  2.00138745  1.3712416 ]
 [ 0.44955446 -0.51792804  1.38199105  3.02017465]
 [-0.36875648  1.43680448 -2.16904652 -1.50881715]
 [-1.27117077  2.06669216 -0.94206739 -4.02568084]
 [ 0.3334998   0.80765162 -1.10316648 -2.46956421]
 [ 0.760867    2.04814916  0.11022919  0.03278453]
 [ 0.04444544 -1.5178885   1.93301018  2.08561156]
 [-1.30561871  2.5464861  -2.64933719 -3.44387113]]
  Bias:
[-0.21307671 -2.92199949 -0.11596912  3.12985721  1.61114867 -0.48396974
 -0.38157642  0.28939568]
Camada 2:
  Pesos:
[[-2.41527334e+00 -2.07360379e+00  2.18466032e+00  2.36936802e+00
   9.12968221e-01  3.55087289e-03 -2.54890145e+00  2.59866888e+00]
 [ 9.45287678e-01 -2.56907773e+00 -1.37697006e+00  2.05142660e+00
   9.87259857e-01 -2.16358107e+00  7.37961576e-01 -3.67103142e+00]
 [ 7.72308455e-01  3.46343233e+00 -1.32875298e+00 -4.62617049e+00
  -2.93025224e+00 -2.19444977e-01  2.41818110e+00 -2.15593293e+00]]
  Bias:
[-0.97824549 